In [3]:
from utils import read_csv_spark, df_analytics, write_csv_spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, sum as spark_sum, count, avg, min, max
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, DoubleType, BooleanType

spark = SparkSession.builder.appName('deteriorations').getOrCreate()

In [2]:
spark = SparkSession.builder.appName('deteriorations').getOrCreate()
events_concat = read_csv_spark('../data/processed/events_concat.csv', spark)
df_analytics(events_concat, 'events_concat')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/11 18:42:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/11 18:42:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/11 18:42:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


events_concat.csv:
+----------+------+-------------------+-------------+--------+--------+--------+----------+
|SUBJECT_ID|ITEMID|          CHARTTIME|        VALUE|VALUENUM|VALUEUOM|    FLAG|EVENT_TYPE|
+----------+------+-------------------+-------------+--------+--------+--------+----------+
|         3| 50820|2101-10-12 16:07:00|         7.39|    7.39|   units|    NULL|       lab|
|         3| 50800|2101-10-12 18:17:00|          ART|    NULL|    NULL|    NULL|       lab|
|         3| 50802|2101-10-12 18:17:00|           -1|      -1|   mEq/L|    NULL|       lab|
|         3| 50804|2101-10-12 18:17:00|           22|      22|   mEq/L|    NULL|       lab|
|         3| 50808|2101-10-12 18:17:00|         0.93|     .93|  mmol/L|abnormal|       lab|
|         3| 50812|2101-10-12 18:17:00|NOT INTUBATED|    NULL|    NULL|    NULL|       lab|
|         3| 50813|2101-10-12 18:17:00|          1.8|     1.8|  mmol/L|    NULL|       lab|
|         3| 50818|2101-10-12 18:17:00|           33|      33

362915756 X 8

For df events_concat:
+----------+------+-------------------+-------------+--------+--------+--------+----------+
|SUBJECT_ID|ITEMID|          CHARTTIME|        VALUE|VALUENUM|VALUEUOM|    FLAG|EVENT_TYPE|
+----------+------+-------------------+-------------+--------+--------+--------+----------+
|         3| 50820|2101-10-12 16:07:00|         7.39|    7.39|   units|    NULL|       lab|
|         3| 50800|2101-10-12 18:17:00|          ART|    NULL|    NULL|    NULL|       lab|
|         3| 50802|2101-10-12 18:17:00|           -1|      -1|   mEq/L|    NULL|       lab|
|         3| 50804|2101-10-12 18:17:00|           22|      22|   mEq/L|    NULL|       lab|
|         3| 50808|2101-10-12 18:17:00|         0.93|     .93|  mmol/L|abnormal|       lab|
|         3| 50812|2101-10-12 18:17:00|NOT INTUBATED|    NULL|    NULL|    NULL|       lab|
|         3| 50813|2101-10-12 18:17:00|          1.8|     1.8|  mmol/L|    NULL|       lab|
|         3| 50818|2101-10-12 18:17:00|    

24/07/11 18:50:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+--------------------+---------+---------+----------+
|summary|        SUBJECT_ID|            ITEMID|             VALUE|            VALUENUM| VALUEUOM|     FLAG|EVENT_TYPE|
+-------+------------------+------------------+------------------+--------------------+---------+---------+----------+
|  count|         362915756|         362915756|         360753051|           175786222|145680415|344905868| 362915756|
|   mean| 31334.91656933462|  74158.1979291001| 78.73736456820652|   70.99966761517543|     NULL|     NULL|      NULL|
| stddev|27031.504199489762|100380.41036958448|2910.2378036451596|   2793.820985714249|     NULL|     NULL|      NULL|
|    min|                 2|                 1|                 !|                 -.1|        #|     NULL|     chart|
|    25%|             11102|               617|               9.0|                 5.0|     NULL|     NULL|      NULL|
|    50%|             22588|              5813| 

362915756 X 8



In [5]:
# Helper function to determin if the value is numerical or not
def is_numerical_udf(value):
    if value is None:
        return False
    try:
        float(value)
        return True
    except ValueError:
        return False

is_numerical = udf(is_numerical_udf, BooleanType())

In [6]:
# add column "is_numerical" using the UDF
events_concat = events_concat.withColumn('IS_NUMERIC', is_numerical(col('VALUE')))

# aggregate counts of numerical values and add them as new columns
events_concat_aggregated = events_concat.groupBy('ITEMID').agg(
    spark_sum(col('IS_NUMERIC').cast('int')).alias('NUM_NUMERIC'),
    (count(col('IS_NUMERIC')) - spark_sum(col('IS_NUMERIC').cast('int'))).alias('NUM_STRING')
)

# create ratio column as well - NUM_NUMERIC / (NUM_NUMERIC + NUM_STRING)
events_concat_aggregated = events_concat_aggregated.withColumn('RATIO', col('NUM_NUMERIC') / (col('NUM_NUMERIC') + col('NUM_STRING')))

df_analytics(events_concat_aggregated, 'events_concat_aggregated')

For df events_concat_aggregated:


+------+-----------+----------+-------------------+
|ITEMID|NUM_NUMERIC|NUM_STRING|              RATIO|
+------+-----------+----------+-------------------+
| 51123|        461|         0|                1.0|
| 51415|          0|        22|                0.0|
| 51393|          0|        21|                0.0|
| 50954|     107114|         6| 0.9999439880507842|
| 51388|          0|       384|                0.0|
| 51454|         83|         0|                1.0|
| 51433|         46|         0|                1.0|
| 51142|          0|        16|                0.0|
| 51243|          0|       554|                0.0|
| 51022|         52|         1| 0.9811320754716981|
| 50847|         48|         0|                1.0|
| 50803|       9242|         4| 0.9995673804888601|
| 51086|          0|       791|                0.0|
| 50963|       6780|       163| 0.9765231168082962|
| 51455|       2612|         0|                1.0|
| 51514|      25116|     76701|0.24667786322519816|
| 51127|    

+-------+------------------+------------------+------------------+-------------------+
|summary|            ITEMID|       NUM_NUMERIC|        NUM_STRING|              RATIO|
+-------+------------------+------------------+------------------+-------------------+
|  count|              8344|              8344|              8344|               8344|
|   mean|56748.839046021094|19719.294343240654|23774.923777564716| 0.4913245404263566|
| stddev| 85260.12830051231| 152156.6984384926|130248.55730370566|0.48525406877997646|
|    min|                 1|                 0|                 0|                0.0|
|    25%|              3120|                 0|                 0|                0.0|
|    50%|              7021|                 1|                10|                0.5|
|    75%|             51081|                42|               319|                1.0|
|    max|            228451|           5161489|           3303151|                1.0|
+-------+------------------+---------------

8344 X 4



In [7]:
write_csv_spark(events_concat_aggregated, '../data/processed/events_is_numeric')

In [5]:
events_is_numeric = read_csv_spark('../data/processed/events_is_numeric', spark)
df_analytics(events_is_numeric, "events_is_numeric")

events_is_numeric:
+------+-----------+----------+-------------------+
|ITEMID|NUM_NUMERIC|NUM_STRING|              RATIO|
+------+-----------+----------+-------------------+
| 51222|     752277|       167| 0.9997780565729808|
| 51351|       4732|         0|                1.0|
| 50810|      89713|         2|   0.99997770718386|
| 51005|        590|         2| 0.9966216216216216|
| 51101|        150|         0|                1.0|
| 51310|          0|        99|                0.0|
| 51479|       2782|      3763|0.42505729564553096|
| 51438|       1366|         2| 0.9985380116959064|
| 51481|          0|       157|                0.0|
| 51434|        546|         0|                1.0|
| 51183|          0|       256|                0.0|
| 51401|          0|         9|                0.0|
| 51097|      10852|        79| 0.9927728478638734|
| 50953|      17229|         0|                1.0|
| 51034|        468|         0|                1.0|
| 51081|        150|         0|              

24/07/12 17:02:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+-------------------+
|summary|            ITEMID|       NUM_NUMERIC|        NUM_STRING|              RATIO|
+-------+------------------+------------------+------------------+-------------------+
|  count|              8344|              8344|              8344|               8344|
|   mean|56748.839046021094|19719.294343240654|23774.923777564716| 0.4913245404263565|
| stddev| 85260.12830051231| 152156.6984384926|130248.55730370567|0.48525406877997646|
|    min|                 1|                 0|                 0|                0.0|
|    25%|              3120|                 0|                 0|                0.0|
|    50%|              7021|                 1|                10|                0.5|
|    75%|             51081|                42|               319|                1.0|
|    max|            228451|           5161489|           3303151|                1.0|
+-------+------------------+---------------

In [13]:
# get events counts associated with numerical value entry ratio
count_total = events_is_numeric.count()
count_ratio_1 = events_is_numeric.filter(col('RATIO') == 1).count()
count_ratio_0 = events_is_numeric.filter(col('RATIO') == 0).count()
print(f"count_total: {count_total}")
print(f"count_ratio_1: {count_ratio_1}")
print(f"count_ratio_0: {count_ratio_0}")

count_ratio_ge_09 = events_is_numeric.filter(col('RATIO') >= 0.9).count()
count_Ratio_ge_05 = events_is_numeric.filter(col('RATIO') >= 0.5).count()
count_ratio_le_01 = events_is_numeric.filter(col('RATIO') < 0.1).count()
count_ratio_ge_05_le_09 = events_is_numeric.filter((col('RATIO') >= 0.5) & (col('RATIO') < 0.9)).count()
print(f"count_ratio_ge_09", count_ratio_ge_09)
print(f"count_Ratio_ge_05", count_Ratio_ge_05)
print(f'count_ratio_le_01', count_ratio_le_01)
print(f"count_ratio_ge_05_le_09", count_ratio_ge_05_le_09)

count_total: 8344
count_ratio_1: 2846
count_ratio_0: 3919
count_ratio_ge_09 3787
count_Ratio_ge_05 4194
count_ratio_le_01 4023
count_ratio_ge_05_le_09 407


In [2]:
# Analytics for all_events_string_df
all_events_string = read_csv_spark('../data/processed/all_events_string_df', spark)
df_analytics(all_events_string, 'all_events_string')

all_events_string_df:
+------+----------+-------------------+-----+--------+--------+----+----------+
|ITEMID|SUBJECT_ID|          CHARTTIME|VALUE|VALUENUM|VALUEUOM|FLAG|EVENT_TYPE|
+------+----------+-------------------+-----+--------+--------+----+----------+
|   427|     11289|2180-12-08 00:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 02:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 04:00:00| Cool|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 06:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-10-29 09:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-11-02 03:00:00| Cool|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-10-29 19:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11305|2116-06-10 11:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11305|2116-06-10 15:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11318

196374702 X 8

For df all_events_string:
+------+----------+-------------------+-----+--------+--------+----+----------+
|ITEMID|SUBJECT_ID|          CHARTTIME|VALUE|VALUENUM|VALUEUOM|FLAG|EVENT_TYPE|
+------+----------+-------------------+-----+--------+--------+----+----------+
|   427|     11289|2180-12-08 00:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 02:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 04:00:00| Cool|    NULL|    NULL|NULL|     chart|
|   427|     11289|2180-12-08 06:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-10-29 09:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-11-02 03:00:00| Cool|    NULL|    NULL|NULL|     chart|
|   427|     11318|2124-10-29 19:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11305|2116-06-10 11:00:00| Warm|    NULL|    NULL|NULL|     chart|
|   427|     11305|2116-06-10 15:00:00| Warm|    NULL|    NULL|NULL|     chart|

196374702 X 8



In [4]:
# udf for word counting
def count_words(value):
    if value is None:
        return 0
    return len(value.split())


In [5]:
count_words_udf = udf(count_words, IntegerType())
all_events_string = all_events_string.withColumn('WORD_COUNT', count_words_udf(all_events_string['VALUE']))

all_events_string.show()

word_count_stats = all_events_string.select(
    count("*").alias("total_rows"),
    avg("word_count").alias("average_word_count"),
    min("word_count").alias("min_word_count"),
    max("word_count").alias("max_word_count")
)

# Show the statistics
word_count_stats.show()

+------+----------+-------------------+-----+--------+--------+----+----------+----------+
|ITEMID|SUBJECT_ID|          CHARTTIME|VALUE|VALUENUM|VALUEUOM|FLAG|EVENT_TYPE|WORD_COUNT|
+------+----------+-------------------+-----+--------+--------+----+----------+----------+
|   427|     11289|2180-12-08 00:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|
|   427|     11289|2180-12-08 02:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|
|   427|     11289|2180-12-08 04:00:00| Cool|    NULL|    NULL|NULL|     chart|         1|
|   427|     11289|2180-12-08 06:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|
|   427|     11318|2124-10-29 09:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|
|   427|     11318|2124-11-02 03:00:00| Cool|    NULL|    NULL|NULL|     chart|         1|
|   427|     11318|2124-10-29 19:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|
|   427|     11305|2116-06-10 11:00:00| Warm|    NULL|    NULL|NULL|     chart|         1|

+----------+------------------+--------------+--------------+
|total_rows|average_word_count|min_word_count|max_word_count|
+----------+------------------+--------------+--------------+
| 196374702| 1.504812188079094|             1|            18|
+----------+------------------+--------------+--------------+

